In [1]:
from dotenv import load_dotenv
load_dotenv()

True

### [Runnable](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.base.Runnable.html)
#### 1. Key Methods
- (a)invoke, (a)batch, (a)stream, astream_log
- built-in-optimizations for Batch, Async
- All methods accept a config argument ex. tags, tracing  

p.s. schematic information of Runnables - use property: input_schema, output_schema, config_schema

#### 2. LCEL and Composition
- LCEL: declarative way to compose runnables into chains
- chain constructed in LCEL way -> have (a)sync, batch, stream support

##### main composition primitives:
1. RunnableSequence:
- invokes runnables sequentially
- runnable -> (output) <=> (input) -> runnable
- example of using operator of RunnableSequence: **a | b | c** 
  
2. RunnableParallel
- invokes runnables cocurrently
- provide same input to each (runnables)  
runnable -> (output) <=> (input) -> runnable  
------------------------<=> (input) -> runnable  
- operate by using **dict** in the sequence ex. {'a':lambda, 'b':lambda}
- or operate by passing dict to Runnable Parallel (???)

In [26]:
# RunnableSequence
from langchain_core.runnables import RunnableLambda

sequence = RunnableLambda(lambda x: x + 1) | RunnableLambda(lambda x: x * 2)
print(sequence.invoke(1))
print(sequence.batch([1,2,3]))

# RunnableParallel
sequence = RunnableLambda(lambda x: x + 1) | {
    'mul_2': RunnableLambda(lambda x: x * 2),
    'mul_5': RunnableLambda(lambda x: x * 5)
}
sequence.invoke(1)

4
[4, 6, 8]


{'mul_2': 4, 'mul_5': 10}

#### 3. Standard Methods
- all methods work for any runnables ex. `retry policy`, add lifecycle listeners, make them configurable
- including runnables contructed by other runnables


In [28]:
# example of retry policy for a runnable

from langchain_core.runnables import RunnableLambda
import random

def add_one(x:int) -> int:
    return x + 1

def buggy_double(y:int) -> int:
    """Buggy cod that will fail 70% of the time"""
    random_num = random.random()
    print("random_num: ", random_num)
    if random_num > 0.3:
        print('This code failed, and will probably be retried!')
        raise ValueError('Triggered buggy code')
    return y * 2

sequence = (
    RunnableLambda(add_one) | 
    RunnableLambda(buggy_double).with_retry( # method for this example
        stop_after_attempt=10,
        wait_exponential_jitter=False
    )
)

print(sequence.input_schema.model_json_schema()) # chain의 가장 첫 input에 대한 title(함수명+input), type
print(sequence.output_schema.model_json_schema()) # chain의 최종 output에 대한 title(함수명+output), type
print()
print(sequence.invoke(2))

{'title': 'add_one_input', 'type': 'integer'}
{'title': 'buggy_double_output', 'type': 'integer'}

random_num:  0.8454297760636902
This code failed, and will probably be retried!
random_num:  0.990978149158021
This code failed, and will probably be retried!
random_num:  0.43558078074659046
This code failed, and will probably be retried!
random_num:  0.5701463823620421
This code failed, and will probably be retried!
random_num:  0.6360043643641371
This code failed, and will probably be retried!
random_num:  0.3206983602446597
This code failed, and will probably be retried!
random_num:  0.6353118116245488
This code failed, and will probably be retried!
random_num:  0.6702551244936716
This code failed, and will probably be retried!
random_num:  0.23846841331781998
6


#### 4. Debugging and Tracing
- It is useful to see intermediate results to debug and trace the chain
- global debug flag: enable debug output for all chains
- passing callbacks to chain: use 'config' of invoke()

In [ ]:
# global debug flag
from langchain_core.globals import set_debug
set_debug(False) # True로 설정할 경우, intput output 진행상황이 자세히 나옴

In [27]:
# passing callbacks to chain
from langchain_core.tracers import ConsoleCallbackHandler

chain = RunnableLambda(lambda x: x + 1) | {
    'mul_2': RunnableLambda(lambda x: x * 2),
    'mul_5': RunnableLambda(lambda x: x * 5)
}
chain.invoke(
    1,
    config={'callbacks': [ConsoleCallbackHandler()]}
)

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "input": 1
}
[chain/start] [chain:RunnableSequence > chain:RunnableLambda] Entering Chain run with input:
{
  "input": 1
}
[chain/end] [chain:RunnableSequence > chain:RunnableLambda] s] Exiting Chain run with output:
{
  "output": 2
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<mul_2,mul_5>] Entering Chain run with input:
{
  "input": 2
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<mul_2,mul_5> > chain:RunnableLambda] Entering Chain run with input:
{
  "input": 2
}
[chain/end] [chain:RunnableSequence > chain:RunnableParallel<mul_2,mul_5> > chain:RunnableLambda] s] Exiting Chain run with output:
{
  "output": 4
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<mul_2,mul_5> > chain:RunnableLambda] Entering Chain run with input:
{
  "input": 2
}
[chain/end] [chain:RunnableSequence > chain:RunnableParallel<mul_2,mul_5> > chain:RunnableLambda] s] Exiting Chain run w

{'mul_2': 4, 'mul_5': 10}